In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import psycopg2

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def GMM(dat, k):
    if k == 1:
        p_class_given_data = np.ones((len(dat), 1))
        means = np.array([np.mean(dat.loc[:, "lat"]), np.mean(dat.loc[:, "lng"])])
        covars = np.cov(np.transpose(dat))
        p_class = np.array([1])

        p_sum = 0
        for i in range(len(dat)):
            p_sum += p_class_given_data[i][0]*(np.square(dat.iloc[i].values-means[0]))
        mean_dist = np.sqrt(p_sum / len(dat.index))
        return p_class_given_data, means, covars, p_class, mean_dist

    """Define the variables we will use in the Algorithm"""
    #This represents Σ_m (covariance matrix of each component) (slide 26)
    covars=np.zeros((k,2,2))

    #This represents μ_m (mean values of each component) (slide 26)
    means=np.zeros((k,2))

    #This represents W_m (weights/likelihood of each component) (slide 26 bottom)
    p_class=np.zeros(k)

    #This represents P(x_i | cluster = m), or Φ(x_i | μ_m, Σ_m) (slide 27)
    #During the Expectation step, we will use this to derive p̂_mi (or p_class_given_data) as shown on slide 27
    #p_class_given_data will be the same shape
    p_data_given_class=np.zeros((len(dat),k))
    p_class_given_data=np.zeros((len(dat),k))


    """Initialize means, covs, p_classes"""
    #initializations of starting points (used to set the initial means below)
    init_idx=np.random.choice(range(len(dat)), size=k, replace=False)

    #Initialize the covariance matrix, the means, and the p_class for each of k components (dims)
    for dim in range(k):
        #Set the cov matrix of each component to the cov of the entire dataset (slide 26)
        covars[dim,:,:]=np.cov(np.transpose(dat))

        #Set initial means to initial chosen data points (slide 26)
        means[dim,:]=dat.iloc[init_idx[dim]]

        #Give each component equal weighting / likelihood to start (slide 26)
        p_class[dim]=1/k

    """Now Iterate"""
    #Now we start our for loop, where-in we'll alernate between: (1) Expecation Step and (2) Maximization Step
    #Note - 50 steps is fine, no requirement to check for convergence on this HW

    for step in range(50):
        """Expectation Step (Slide 27)"""
        for row_i in range(len(dat.index)):
            for component in range(k):
                p_data_given_class[row_i][component] = stats.multivariate_normal(means[component], covars[component]).cdf(dat.iloc[row_i].values.tolist())*p_class[component]
            for component in range(k):
                p_class_given_data[row_i][component] = p_data_given_class[row_i][component]/sum(p_data_given_class[row_i])
        """Maximization Step (Slide 29-30)"""
        for component in range(k):
            n_m = sum(p_class_given_data[:, component])
            p_class[component] = n_m/len(dat.index)
            weighted_values = [[p_class_given_data[idx_x][component]*dat.iloc[idx_x]["lat"], p_class_given_data[idx_x][component]*dat.iloc[idx_x]["lng"]] for idx_x in range(len(dat.index))]
            means[component, :] = list(np.sum(weighted_values, axis=0) / n_m)

            covar = [[0, 0],[0, 0]]
            x = dat.loc[:, "lat"]
            y = dat.loc[:, "lng"]
            for i in range(k):
                covar[0][0] += (x[i]-means[component][0])*(x[i]-means[component][0])
                covar[0][1] += (x[i]-means[component][0])*(y[i]-means[component][1])
                covar[1][0] += (y[i]-means[component][1])*(x[i]-means[component][0])
                covar[1][1] += (y[i]-means[component][1])*(y[i]-means[component][1])

            covars[component, :, :] = list(np.array(covar)/n_m)
    mean_dist = 0
    """Once we're done with our for loop, we compute the mean dist (This formula is given in the HW below in Part B)"""

    p_sum = 0
    for i in range(len(dat.index)):
        for component in range(k):
            p_sum += p_class_given_data[i][component]*(np.square(dat.iloc[i].values-means[component]))
    mean_dist = np.sqrt(p_sum / len(dat.index))
    return p_class_given_data, means, covars, p_class, mean_dist

In [ ]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432",
)

cur = conn.cursor()